# bert 微调
### 文本分类
    *针对[CLS]后接分类层
### 语义匹配
    *可当做二分类
    *使用孪生网络（暂不作介绍）
    
### 序列标注
    *针对token,外加lstm+crf
### QA问答
    *训练start、end位置

In [5]:
import torch
from torch import nn
import torch.nn.functional as F
from transformers import BertPreTrainedModel,BertModel

###  1、文本分类
#### 1.1 模型代码
#### 1.2 注意事项

In [7]:
#bert + textcnn
class Conv1d(nn.Module):
    def __init__(self,in_channel,out_channel,filter_sizes):
        super(Conv1d,self).__init__()
        self.convs=nn.ModuleList([
            nn.Conv1d(in_channels=in_channel,
                     out_channels=out_channel,
                     kernel_size=fs)
            for fs in filter_sizes
        ])  #每一次conv1d通道可以不同
        self.init_params()
        
    def init_params(self):
        for n in self.convs:
            nn.init.xavier_uniform_(m.weight.data)  #初始化
            nn.init.constant_(m.bias,data,0.1)  #初始化
        
    
    def forward(self,x):
        return [F.relu(conv(x)) for conv in self.convs]

class BertCNN(BertPreTrainedModel):
    def __init__(self,config,num_labels,n_filters,filter_sizes):
        super(BertCNN).__init__(config)
        self.num_labels=num_labels
        self.bert=BertModel(config)
        self.dropout=nn.Dropout(config.drop_prob)
        #Textcnn
        self.convs=Conv1d(config.hidden_size,n_filters,filter_sizes)
        self.classifier=nn.Linear(len(filter_sizes)*n_filters,num_labels)
        #lstm
        #self.rnn = nn.LSTM(config.hidden_size, rnn_hidden_size, num_layers,bidirectional=bidirectional, batch_first=True, dropout=dropout)
        #self.classifier = nn.Linear(rnn_hidden_size * 2, num_labels)
        
        #attention
        #self.classifier = nn.Linear(config.hidden_size, num_labels)
        #self.W_w = nn.Parameter(torch.Tensor(config.hidden_size, config.hidden_size))
        #self.u_w = nn.Parameter(torch.Tensor(config.hidden_size, 1))
        #nn.init.uniform_(self.W_w, -0.1, 0.1)
        #nn.init.uniform_(self.u_w, -0.1, 0.1)
        
        self.apply(self.init_bert_weights)
    
        
    def forward(self,input_ids,token_type_ids=None,attention_mask=None,labels=None):
        """
        Args:
            input_ids: 词对应的 id
            token_type_ids: 区分句子，0 为第一句，1表示第二句
            attention_mask: 区分 padding 与 token， 1表示是token，0 为padding
        """
        encoded_layers, _ = self.bert(
            input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        # encoded_layers: [batch_size, seq_len, bert_dim=768]
        
        encoded_layers = self.dropout(encoded_layers)
        """
        Textcnn（3选1）
        # 对encoded_layers做维度调整
        # 调用conv层
        # 图中所示采用最大池化融合
        """
        encoded_layers = encoded_layers.permute(0, 2, 1)
        # encoded_layers: [batch_size, bert_dim=768, seq_len]

        conved = self.convs(encoded_layers) ##[batch_size,768,seq_len]--nn.Conv()--> n_filters个[batch_size,out_channel,xxx]
        # conved 是一个列表， conved[0]: [batch_size, filter_num, *]

        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2)
                  for conv in conved]   ##[batch_size,out_channel,1]--->[batch_size,out_channel]  n_filters个
        # pooled 是一个列表， pooled[0]: [batch_size, filter_num]
        cat = self.dropout(torch.cat(pooled, dim=1))
        # cat: [batch_size, filter_num * len(filter_sizes)]
        '''
        bert+rnn可以替换上面的代码(3选1)
         _, (hidden, cell) = self.rnn(encoded_layers)
         cat = self.dropout(
            torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))  # 连接最后一层的双向输出
        '''
        
        '''
        bert_attention可以替换上面的代码（3选1）
        score = torch.tanh(torch.matmul(encoded_layers, self.W_w))
        # score: [batch_size, seq_len, bert_dim]

        attention_weights = F.softmax(torch.matmul(score, self.u_w), dim=1)
        # attention_weights: [batch_size, seq_len, 1]

        scored_x = encoded_layers * attention_weights
        # scored_x : [batch_size, seq_len, bert_dim]

        cat = torch.sum(scored_x, dim=1)
        # cat: [batch_size, bert_dim=768]
        
        '''
        
        logits = self.classifier(cat)
        # logits: [batch_size, output_dim]

        if labels is not None:
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return loss
        else:
            return logits
    
